In [ ]:
from groq import Groq
import os
import base64
import time

VISION_MODEL = "meta-llama/llama-4-scout-17b-16e-instruct" 

# --- FONCTION POUR ENCODER L'IMAGE ---
def encode_image(image_path):
    """Encodes a local file to a Base64 string."""
    try:
        with open(image_path, "rb") as image_file:
            return base64.b64encode(image_file.read()).decode('utf-8')
    except FileNotFoundError:
        print(f"Erreur: Le fichier n'a pas été trouvé à l'emplacement: {image_path}")
        return None
    except Exception as e:
        print(f"Erreur lors de l'encodage de l'image: {e}")
        return None

# --- VÉRIFICATION ET ENCODAGE ---
def check_file_exists(file_path):
    """Vérifie si le fichier existe à l'emplacement donné."""
    return os.path.isfile(file_path)

def generate_base64_from_local(file_path):
    """Génère une chaîne Base64 à partir d'un fichier local."""
    if not check_file_exists(file_path):
        print(f"Erreur: Le fichier n'existe pas à l'emplacement: {file_path}")
        return None
    return encode_image(file_path)

def generate_answer():
    local_file_path = r"C:\Users\ihadi\Downloads\téléchargement (1).jpeg" 
    base64_image = generate_base64_from_local(local_file_path)

    if not base64_image:
        exit()

    # Format Base64 obligatoire pour l'API
    base64_url = f"data:image/jpeg;base64,{base64_image}"

    # --- APPEL À L'API GROQ ---
    client = Groq(api_key=os.getenv("GROQ_API_KEY"))

    # Note: Je remplace le modèle par le modèle Vision Groq supporté (Llama 3.2 Vision)
    # Si vous avez un accès spécial, vous pouvez remettre votre nom de modèle.
    

    start_time = time.time()
    print(f"Envoi du fichier local ({local_file_path}) à Groq...")

    completion = client.chat.completions.create(
        model=VISION_MODEL,
        messages=[
            {
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        "text": "I want you to generate a Json file of the document. Json file : "
                    },
                    {
                        "type": "image_url",
                        "image_url": {
                            # Utilisation de la chaîne Base64 encodée
                            "url": base64_url
                        }
                    }
                ]
            }
        ],
        temperature=1,
        max_completion_tokens=1024,
        top_p=1,
        stream=True,
        stop=None
    )

    print("\n--- RÉSULTAT (Généré par Groq) ---")
    for chunk in completion:
        print(chunk.choices[0].delta.content or "", end="")

    end_time = time.time()
    print(f"\n\nTemps d'exécution total: {end_time - start_time:.2f} secondes")

Envoi du fichier local (C:\Users\ihadi\Downloads\téléchargement (1).jpeg) à Groq...

--- RÉSULTAT (Généré par Groq) ---
Here is a JSON representation of the receipt document:


```json
{
  "store": "Lidl",
  "receiptType": "Ticket de caisse",
  "date": null,
  "items": [
    {
      "name": "carottes",
      "price": 0.69,
      "unit": "kg",
      "quantity": 0.338
    },
    {
      "name": "poivrons",
      "price": 2.99,
      "unit": null,
      "quantity": 1
    },
    {
      "name": "oignons",
      "price": 0.69,
      "unit": "kg",
      "quantity": 0.228
    },
    {
      "name": "aubergines",
      "price": 1.75,
      "unit": null,
      "quantity": 1
    },
    {
      "name": "tomates",
      "price": 1.29,
      "unit": null,
      "quantity": 1
    },
    {
      "name": "Fraisas",
      "price": 2.99,
      "unit": null,
      "quantity": 1
    },
    {
      "name": "Petit crevettes",
      "price": 6.99,
      "unit": null,
      "quantity": 1
    },
    {
      "n

In [8]:
# # Load model directly
# from transformers import AutoProcessor, AutoModelForVision2Seq
# import torch 

# processor = AutoProcessor.from_pretrained("C:\\Users\\ihadi\\Desktop\\Semestre 9\\LLM and Gen AI\\Projet\\IDP_GenAI_Project\\models\\Fara-7B", 
#                                           torch_dtype=torch.float16,
#                                           device_map="auto")

# model = AutoModelForVision2Seq.from_pretrained("C:\\Users\\ihadi\\Desktop\\Semestre 9\\LLM and Gen AI\\Projet\\IDP_GenAI_Project\\models\\Fara-7B",
#                                                torch_dtype=torch.float16,
#                                                device_map="auto")

In [9]:
from transformers import AutoModel

model = AutoModel.from_pretrained("nvidia/NVIDIA-Nemotron-Parse-v1.1-TC", 
                                  trust_remote_code=True, 
                                  dtype="auto",
                                  cache_dir=LOCAL_MODEL_PATH)

c:\Users\ihadi\Desktop\Semestre 9\LLM and Gen AI\Projet\IDP_GenAI_Project\.venv\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\ihadi\Desktop\Semestre 9\LLM and Gen AI\Projet\IDP_GenAI_Project\models\models--nvidia--NVIDIA-Nemotron-Parse-v1.1-TC. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
A new ver

In [1]:
from transformers import AutoProcessor, Florence2ForConditionalGeneration
import torch
from PIL import Image
import os 
import time

# --- CHEMINS (Utilisez le chemin absolu pour une fiabilité maximale) ---
LOCAL_MODEL_PATH = r"C:\Users\ihadi\Desktop\Semestre 9\LLM and Gen AI\Projet\IDP_GenAI_Project\models\Florence-2-base" 
local_file_path = r"C:\Users\ihadi\Downloads\téléchargement (1).jpeg"

try:
    image = Image.open(local_file_path).convert("RGB")
except FileNotFoundError:
    print(f"Erreur: Le fichier image n'a pas été trouvé à l'emplacement: {local_file_path}")
    exit()

print("Démarrage du chargement du modèle sur CPU...")
start_time = time.time()

# --- CHARGEMENT : device_map="cpu" est CRUCIAL ---
model = Florence2ForConditionalGeneration.from_pretrained(
    LOCAL_MODEL_PATH, 
    trust_remote_code=True, 
    torch_dtype=torch.float32
)

processor = AutoProcessor.from_pretrained(
    LOCAL_MODEL_PATH, 
    trust_remote_code=True
)

# --- INFÉRENCE : La solution de contournement du Token ---

# 1. Obtenir le token image exact et le token de début de séquence (BOS)
IMAGE_TOKEN = processor.tokenizer.additional_special_tokens[0]
BOS_TOKEN = processor.tokenizer.bos_token

# 2. Construire la chaîne complète : BOS + Image Token + Instruction
# L'ajout du BOS token rend souvent l'inférence plus stable.
prompt_instructionnel = "Generate a JSON of the document." 
full_prompt = BOS_TOKEN + IMAGE_TOKEN + prompt_instructionnel 

print(f"Prompt final : {full_prompt}")

# 3. Utiliser le processeur pour l'image, mais le tokenizer pour le texte
# Le processeur doit voir l'image et le texte dans le bon ordre.
# Le processeur Florence-2 gère l'encodage du texte pour nous.
inputs = processor(
    text=full_prompt, 
    images=image, 
    return_tensors="pt"
)

# Déplacement des inputs sur CPU
inputs = {k: v.to(model.device) for k, v in inputs.items()}

print("Génération de la réponse...")
generated_ids = model.generate(
    **inputs, 
    max_new_tokens=512, 
    do_sample=False
)

# --- DÉCODAGE ---
result = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
end_time = time.time()

print("\n--- RÉSULTAT ---")
print(result)
print(f"\nTemps de génération total: {end_time - start_time:.2f} secondes.")

c:\Users\ihadi\Desktop\Semestre 9\LLM and Gen AI\Projet\IDP_GenAI_Project\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
`torch_dtype` is deprecated! Use `dtype` instead!


Démarrage du chargement du modèle sur CPU...


Some weights of the model checkpoint at C:\Users\ihadi\Desktop\Semestre 9\LLM and Gen AI\Projet\IDP_GenAI_Project\models\Florence-2-base were not used when initializing Florence2ForConditionalGeneration: ['image_pos_embed.column_embeddings.weight', 'image_pos_embed.row_embeddings.weight', 'image_proj_norm.bias', 'image_proj_norm.weight', 'image_projection', 'language_model.final_logits_bias', 'language_model.model.decoder.embed_positions.weight', 'language_model.model.decoder.layernorm_embedding.bias', 'language_model.model.decoder.layernorm_embedding.weight', 'language_model.model.decoder.layers.0.encoder_attn.k_proj.bias', 'language_model.model.decoder.layers.0.encoder_attn.k_proj.weight', 'language_model.model.decoder.layers.0.encoder_attn.out_proj.bias', 'language_model.model.decoder.layers.0.encoder_attn.out_proj.weight', 'language_model.model.decoder.layers.0.encoder_attn.q_proj.bias', 'language_model.model.decoder.layers.0.encoder_attn.q_proj.weight', 'language_model.model.decod

Prompt final : <s><od>Generate a JSON of the document.
Génération de la réponse...


ValueError: Image features and image tokens do not match: tokens: 0, features 577

In [1]:
# WARNING: This snippet is not yet compatible with SageMaker version >= 3.0.0.
# To use this snippet, install a compatible version:
# pip install 'sagemaker<3.0.0'
import json
import sagemaker
import boto3
from sagemaker.huggingface import HuggingFaceModel, get_huggingface_llm_image_uri

try:
	role = sagemaker.get_execution_role()
except ValueError:
	iam = boto3.client('iam')
	role = iam.get_role(RoleName='sagemaker_execution_role')['Role']['Arn']

# Hub Model configuration. https://huggingface.co/models
hub = {
	'HF_MODEL_ID':'microsoft/Fara-7B',
	'SM_NUM_GPUS': json.dumps(1)
}



# create Hugging Face Model Class
huggingface_model = HuggingFaceModel(
	image_uri=get_huggingface_llm_image_uri("huggingface",version="3.3.6"),
	env=hub,
	role=role, 
)

# deploy model to SageMaker Inference
predictor = huggingface_model.deploy(
	initial_instance_count=1,
	instance_type="ml.g5.2xlarge",
	container_startup_health_check_timeout=300,
  )
  
# send request
predictor.predict({
	"inputs": "Hi, what can you help me with?",
})

sagemaker.config INFO - Not applying SDK defaults from location: C:\ProgramData\sagemaker\sagemaker\config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: C:\Users\ihadi\AppData\Local\sagemaker\sagemaker\config.yaml


c:\Users\ihadi\Desktop\Semestre 9\LLM and Gen AI\Projet\IDP_GenAI_Project\sagemaker_env\lib\site-packages\boto3\compat.py:89: PythonDeprecationWarning: Boto3 will no longer support Python 3.9 starting April 29, 2026. To continue receiving service updates, bug fixes, and security updates please upgrade to Python 3.10 or later. More information can be found here: https://aws.amazon.com/blogs/developer/python-support-policy-updates-for-aws-sdks-and-tools/
  warnings.warn(warning, PythonDeprecationWarning)


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:10                                                                                   │
│                                                                                                  │
│    7 from sagemaker.huggingface import HuggingFaceModel, get_huggingface_llm_image_uri           │
│    8                                                                                             │
│    9 try:                                                                                        │
│ ❱ 10 │   role = sagemaker.get_execution_role()                                                   │
│   11 except ValueError:                                                                          │
│   12 │   iam = boto3.client('iam')                                                               │
│   13 │   role = iam.get_role(RoleName='sagemaker_execution_role')['Role']['Arn']                 │
│                                                                                                  │
│ c:\Users\ihadi\Desktop\Semestre 9\LLM and Gen                                                    │
│ AI\Projet\IDP_GenAI_Project\sagemaker_env\lib\site-packages\sagemaker\session.py:8273 in         │
│ get_execution_role                                                                               │
│                                                                                                  │
│   8270 │   │   (str): The role ARN                                                               │
│   8271 │   """                                                                                   │
│   8272 │   if not sagemaker_session:                                                             │
│ ❱ 8273 │   │   sagemaker_session = Session()                                                     │
│   8274 │   arn = sagemaker_session.get_caller_identity_arn()                                     │
│   8275 │                                                                                         │
│   8276 │   if ":role/" in arn:                                                                   │
│                                                                                                  │
│ c:\Users\ihadi\Desktop\Semestre 9\LLM and Gen                                                    │
│ AI\Projet\IDP_GenAI_Project\sagemaker_env\lib\site-packages\sagemaker\session.py:265 in __init__ │
│                                                                                                  │
│    262 │   │   self.lambda_client = None                                                         │
│    263 │   │   self.settings = settings if settings else SessionSettings()                       │
│    264 │   │                                                                                     │
│ ❱  265 │   │   self._initialize(                                                                 │
│    266 │   │   │   boto_session=boto_session,                                                    │
│    267 │   │   │   sagemaker_client=sagemaker_client,                                            │
│    268 │   │   │   sagemaker_runtime_client=sagemaker_runtime_client,                            │
│                                                                                                  │
│ c:\Users\ihadi\Desktop\Semestre 9\LLM and Gen                                                    │
│ AI\Projet\IDP_GenAI_Project\sagemaker_env\lib\site-packages\sagemaker\session.py:293 in          │
│ _initialize                                                                                      │
│                                                                                                  │
│    290 │   │                                                                                     │
│    291 │   │   self._region_name = self.boto_session.region